# Covid-19 (Coronavirus): Data Extract, Transform, Load (ETL) Notebook
This notebook pulls data from various sources, performs data wrangling to get it into more usable formats, and exports to CSV for use in other notebooks, as well as for future reference.  

Feeds into the Covid19_Analytics notebook.

## Data Sources

### Primary Data Source: Johns Hopkins CSSE Data Repository 
- Link: https://github.com/CSSEGISandData/COVID-19  
- **Live data:**
    - Countries (cases, deaths, recoveries, active)
    - US States (cases, deaths)
    - US County (cases, deaths)
- **Historic time series data:**
    - Countries (cases, deaths, recoveries, active)
    - US States (cases, deaths)
    - US Counties (cases, deaths)

### US State Testing and Hospitalizations: Covid Tracking Project 
- Link: https://covidtracking.com/data/ 
- **Live data:**
    - US State testing and hospitalization, ICU stats
- **Historic time series data:**
    - US State testing and hospitalization, ICU stats

## Import Libraries

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from datetime import datetime
import os

## Get the Datasets

### Dataset 1: John Hopkins

Live data is stored in files named MM-DD-YYYY.csv, so get the current date for live data updates.  
https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data/csse_covid_19_daily_reports

#### 1A: Pull the Data from JH's github repo, immediately export as 'orig'

In [2]:
# Create the filename to load based on today's date

day = str(datetime.now().day) if datetime.now().day >= 10 else "0" + str(datetime.now().day)
month = str(datetime.now().month) if datetime.now().month >= 10 else "0" + str(datetime.now().month)
year = str(datetime.now().year)

jh_live_date = month + "-" + day + "-" + year
jh_live_date

'01-02-2021'

In [3]:
#  First try to pull today's data, but if it's not available yet, pull yesterday's data
try:
    fileURL = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/" + jh_live_date + ".csv"
    jh_live_global = pd.read_csv(fileURL)
except:
    day = str(datetime.now().day-1) if datetime.now().day-1 >= 10 else "0" + str(datetime.now().day-1)
    jh_live_date = month + "-" + day + "-" + year
    fileURL = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/" + jh_live_date + ".csv"
    jh_live_global = pd.read_csv(fileURL)
    
jh_live_date

'01-02-2021'

In [4]:
# Check out the first x rows to make sure the data loaded correctly
jh_live_global.head()

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incident_Rate,Case_Fatality_Ratio
0,NaN,NaN,NaN,Afghanistan,2021-01-03 05:22:52,33.93911,67.709953,51526,2191,41727,5.0,Afghanistan,0.303121,4.252222
1,NaN,NaN,NaN,Albania,2021-01-03 05:22:52,41.15330,20.168300,58991,1190,34353,23448.0,Albania,2049.864480,2.017257
2,NaN,NaN,NaN,Algeria,2021-01-03 05:22:52,28.03390,1.659600,100159,2769,67611,29779.0,Algeria,228.407338,2.764604
3,NaN,NaN,NaN,Andorra,2021-01-03 05:22:52,42.50630,1.521800,8166,84,7463,619.0,Andorra,10568.821588,1.028655
4,NaN,NaN,NaN,Angola,2021-01-03 05:22:52,-11.20270,17.873900,17608,407,11189,6012.0,Angola,53.574686,2.311449


Now immediately export the original jh dataset before making any changes

In [5]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_live_global_orig.csv')
jh_live_global.to_csv(filename, index=False)

**Also pull the global historic time series data.  Note that it won't have live data from today until close to midnight PT.**

In [6]:
jh_hist_global_cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
jh_hist_global_cases.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,12/24/20,12/25/20,12/26/20,12/27/20,12/28/20,12/29/20,12/30/20,12/31/20,1/1/21,1/2/21
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,50655,50810,50886,51039,51280,51350,51405,51526,51526,51526
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,54827,55380,55755,56254,56572,57146,57727,58316,58316,58991
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,97007,97441,97857,98249,98631,98988,99311,99610,99897,100159


In [7]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_cases_orig.csv')
jh_hist_global_cases.to_csv(filename, index=False)

In [8]:
jh_hist_global_deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')
jh_hist_global_deaths.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,12/24/20,12/25/20,12/26/20,12/27/20,12/28/20,12/29/20,12/30/20,12/31/20,1/1/21,1/2/21
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,2126,2139,2149,2160,2174,2179,2181,2191,2191,2191
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,1125,1134,1143,1153,1164,1170,1174,1181,1181,1190
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,2705,2716,2722,2728,2737,2745,2751,2756,2762,2769


In [9]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_deaths_orig.csv')
jh_hist_global_deaths.to_csv(filename, index=False)

In [10]:
jh_hist_global_recovered = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv')
jh_hist_global_recovered.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,12/24/20,12/25/20,12/26/20,12/27/20,12/28/20,12/29/20,12/30/20,12/31/20,1/1/21,1/2/21
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,40359,40444,40784,41096,41441,41543,41612,41727,41727,41727
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,30276,30790,31181,31565,32122,32700,33185,33634,33634,34353
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,64777,65144,65505,65862,66214,66550,66855,67127,67395,67611


In [11]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_recovered_orig.csv')
jh_hist_global_recovered.to_csv(filename, index=False)

In [12]:
jh_hist_uscounties_cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv')
jh_hist_uscounties_cases.head(3)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,12/24/20,12/25/20,12/26/20,12/27/20,12/28/20,12/29/20,12/30/20,12/31/20,1/1/21,1/2/21
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,3942,3990,3999,4029,4065,4105,4164,4190,4239,4268
1,84001003,US,USA,840,1003.0,Baldwin,Alabama,US,30.727750,-87.722071,...,12521,12666,12708,12825,12962,13172,13392,13601,13823,13955
2,84001005,US,USA,840,1005.0,Barbour,Alabama,US,31.868263,-85.387129,...,1390,1396,1398,1406,1417,1462,1492,1514,1517,1528


In [13]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_uscounties_cases_orig.csv')
jh_hist_uscounties_cases.to_csv(filename, index=False)

In [14]:
jh_hist_uscounties_deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv')
jh_hist_uscounties_deaths.head(3)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,12/24/20,12/25/20,12/26/20,12/27/20,12/28/20,12/29/20,12/30/20,12/31/20,1/1/21,1/2/21
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,46,46,46,47,47,47,48,48,50,50
1,84001003,US,USA,840,1003.0,Baldwin,Alabama,US,30.727750,-87.722071,...,151,151,151,152,152,156,160,161,169,169
2,84001005,US,USA,840,1005.0,Barbour,Alabama,US,31.868263,-85.387129,...,32,32,32,32,32,32,32,32,33,33


In [15]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_uscounties_deaths_orig.csv')
jh_hist_uscounties_deaths.to_csv(filename, index=False)

#### 1B: Wrangle the data into more usable format

For the JH live global data, it will be useful to group / subtotal by Country, by US State, and by US County.  

For the historic time series data, these need to be rearranged so that the date is in 
datetime format and pivoted so the date is the index of the DataFrame.

In [16]:
jh_live_global[jh_live_global['Country_Region']=='US'].sort_values('Province_State').head(3)

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incident_Rate,Case_Fatality_Ratio
649,1001.0,Autauga,Alabama,US,2021-01-03 05:22:52,32.539527,-86.644082,4268,50,0,4218.0,"Autauga, Alabama, US",7639.299075,1.171509
686,1075.0,Lamar,Alabama,US,2021-01-03 05:22:52,33.779950,-88.096680,1069,15,0,1054.0,"Lamar, Alabama, US",7743.571170,1.403181
687,1077.0,Lauderdale,Alabama,US,2021-01-03 05:22:52,34.901719,-87.656247,6613,70,0,6543.0,"Lauderdale, Alabama, US",7131.533824,1.058521


##### 1B-1: Pivot Table on the JH live global data, grouped / subtotalled by **Country**

In [17]:
jh_live_countries = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Confirmed', 'Deaths', 'Recovered'], aggfunc=np.sum)

In [18]:
jh_live_countries.sort_values('Confirmed', ascending=False).head(3)

,Confirmed,Deaths,Recovered
Country_Region,,,
US,20427780,350186,0
India,10323965,149435,9927310
Brazil,7716405,195725,6893256


In [19]:
jh_live_countries['Active'] = jh_live_countries['Confirmed'] - jh_live_countries['Deaths'] - jh_live_countries['Recovered']

In [20]:
jh_live_countries['Last_Update'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Last_Update'], aggfunc=np.max)
jh_live_countries['Lat'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Lat'], aggfunc=np.mean)
jh_live_countries['Long'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Long_'], aggfunc=np.mean)

In [21]:
jh_live_countries.sort_values('Confirmed', ascending=False).head(5)

,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long
Country_Region,,,,,,,
US,20427780,350186,0,20077594,2021-01-03 05:22:52,37.936303,-91.379001
India,10323965,149435,9927310,247220,2021-01-03 05:22:52,23.088275,81.806127
Brazil,7716405,195725,6893256,627424,2021-01-03 05:22:52,-12.669522,-48.480493
Russia,3179898,57235,2572279,550384,2021-01-03 05:22:52,54.546312,62.120860
France,2700480,65048,201095,2434337,2021-01-03 05:22:52,8.314863,-21.695174


##### 1B-2: Pivot Table on the JH live global data, grouped / subtotalled by **US State**

In [22]:
jh_live_usstates = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Confirmed', 'Deaths', 'Recovered'], 
                                    aggfunc=np.sum)

In [23]:
jh_live_usstates.sort_values('Confirmed', ascending=False).head(3)

,Confirmed,Deaths,Recovered
Province_State,,,
California,2388557,26542,0
Texas,1793453,28338,0
Florida,1354833,21890,0


In [24]:
jh_live_usstates['Active'] = jh_live_usstates['Confirmed'] - jh_live_usstates['Deaths'] - jh_live_usstates['Recovered']

In [25]:
jh_live_usstates['Last_Update'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Last_Update'], 
                                    aggfunc=np.max)
jh_live_usstates['Lat'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Lat'], 
                                    aggfunc=np.mean)
jh_live_usstates['Long'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Long_'], 
                                    aggfunc=np.mean)

In [26]:
jh_live_usstates.sort_values('Confirmed', ascending=False).head(5)

,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long
Province_State,,,,,,,
California,2388557,26542,0,2362015,2021-01-03 05:22:52,37.843962,-120.728594
Texas,1793453,28338,0,1765115,2021-01-03 05:22:52,31.660643,-98.653069
Florida,1354833,21890,0,1332943,2021-01-03 05:22:52,28.940755,-82.700744
New York,1011665,38273,0,973392,2021-01-03 05:22:52,42.544151,-75.474183
Illinois,975352,18214,0,957138,2021-01-03 05:22:52,39.843618,-89.178330


##### 1B-3: Pivot Table on the JH live global data, grouped / subtotalled by **US County**

In [27]:
jh_live_uscounties = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Confirmed', 'Deaths', 'Recovered'], 
                                    aggfunc=np.sum)

In [28]:
jh_live_uscounties.sort_values('Confirmed', ascending=False).head(5)

,,Confirmed,Deaths,Recovered
Province_State,Admin2,,,
California,Los Angeles,806210,10682,0
Illinois,Cook,397942,8359,0
Arizona,Maricopa,331233,5239,0
Florida,Miami-Dade,304187,4239,0
Texas,Harris,243059,3424,0


In [29]:
jh_live_uscounties['Active'] = jh_live_uscounties['Confirmed'] - jh_live_uscounties['Deaths'] - jh_live_uscounties['Recovered']


In [30]:
jh_live_uscounties['Last_Update'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Last_Update'], 
                                    aggfunc=np.max)
jh_live_uscounties['Lat'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Lat'], 
                                    aggfunc=np.mean)
jh_live_uscounties['Long'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Long_'], 
                                    aggfunc=np.mean)
jh_live_uscounties['FIPS'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['FIPS'], 
                                    aggfunc=np.max)

In [31]:
jh_live_uscounties.sort_values('Confirmed', ascending=False).head(5)

,,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long,FIPS
Province_State,Admin2,,,,,,,,
California,Los Angeles,806210,10682,0,795528,2021-01-03 05:22:52,34.308284,-118.228241,6037.0
Illinois,Cook,397942,8359,0,389583,2021-01-03 05:22:52,41.841448,-87.816588,17031.0
Arizona,Maricopa,331233,5239,0,325994,2021-01-03 05:22:52,33.348359,-112.491815,4013.0
Florida,Miami-Dade,304187,4239,0,299948,2021-01-03 05:22:52,25.611236,-80.551706,12086.0
Texas,Harris,243059,3424,0,239635,2021-01-03 05:22:52,29.858649,-95.393395,48201.0


##### 1B-4: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [32]:
# First, isolate the date columns of the JH historic time series data to be used in the following 3 Pivot Tables
date_columns = jh_hist_global_cases.iloc[:, 4:].columns

In [33]:
jh_hist_countries_cases = pd.pivot_table(jh_hist_global_cases, index=['Country/Region'], 
                                            values=date_columns, aggfunc=np.sum)
jh_hist_countries_cases = jh_hist_countries_cases.transpose()
jh_hist_countries_cases.index = pd.to_datetime(jh_hist_countries_cases.index)
jh_hist_countries_cases = jh_hist_countries_cases.sort_index()

In [34]:
jh_hist_countries_cases.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2020-12-31,51526,58316,99610,8049,17553,159,1625514,159409,28425,360815,...,2496231,19119,77060,1,113558,1465,138004,2099,20725,13867
2021-01-01,51526,58316,99897,8117,17568,159,1629594,159738,28460,362911,...,2549689,19753,77182,1,113884,1474,139223,2101,20997,14084
2021-01-02,51526,58991,100159,8166,17608,159,1634834,159798,28484,364302,...,2607542,20275,77182,1,114083,1482,140287,2101,21230,14491


##### 1B-5: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **Country**

In [35]:
jh_hist_countries_deaths = pd.pivot_table(jh_hist_global_deaths, index=['Country/Region'], values=date_columns, aggfunc=np.sum)
jh_hist_countries_deaths = jh_hist_countries_deaths.transpose()
jh_hist_countries_deaths.index = pd.to_datetime(jh_hist_countries_deaths.index)
jh_hist_countries_deaths = jh_hist_countries_deaths.sort_index()

In [36]:
jh_hist_countries_deaths.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2020-12-31,2191,1181,2756,84,405,5,43245,2823,909,6222,...,73622,181,614,0,1028,35,1400,610,388,363
2021-01-01,2191,1181,2762,84,405,5,43319,2828,909,6261,...,74237,193,614,0,1030,35,1418,610,390,369
2021-01-02,2191,1190,2769,84,407,5,43375,2836,909,6275,...,74682,201,614,0,1032,35,1446,610,392,377


##### 1B-6: Pivot Table on the JH historic **recoveries**, grouped / subtotalled by **Country**

In [37]:
jh_hist_countries_recovered = pd.pivot_table(jh_hist_global_recovered, index=['Country/Region'], values=date_columns, aggfunc=np.sum)
jh_hist_countries_recovered = jh_hist_countries_recovered.transpose()
jh_hist_countries_recovered.index = pd.to_datetime(jh_hist_countries_recovered.index)
jh_hist_countries_recovered = jh_hist_countries_recovered.sort_index()

In [38]:
jh_hist_countries_recovered.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2020-12-31,41727,33634,67127,7432,11044,148,1426676,142801,25762,332952,...,5479,13468,74943,1,107583,1325,117183,1394,18660,11250
2021-01-01,41727,33634,67395,7463,11146,148,1426676,143355,25770,334901,...,5679,13990,75244,1,107901,1325,118926,1396,18773,11347
2021-01-02,41727,34353,67611,7463,11189,148,1447092,143640,25778,336922,...,5744,14592,75244,1,108086,1337,120403,1396,18942,11363


##### 1B-7: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [39]:
jh_hist_uscounties_cases.sort_values('Province_State').head()

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,12/24/20,12/25/20,12/26/20,12/27/20,12/28/20,12/29/20,12/30/20,12/31/20,1/1/21,1/2/21
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,3942,3990,3999,4029,4065,4105,4164,4190,4239,4268
37,84001075,US,USA,840,1075.0,Lamar,Alabama,US,33.779950,-88.096680,...,997,1007,1009,1012,1017,1025,1037,1056,1065,1069
38,84001077,US,USA,840,1077.0,Lauderdale,Alabama,US,34.901719,-87.656247,...,5996,6048,6039,6117,6162,6230,6371,6478,6566,6613
39,84001079,US,USA,840,1079.0,Lawrence,Alabama,US,34.520415,-87.310695,...,1907,1921,1924,1939,1958,1978,2005,2044,2082,2102
40,84001081,US,USA,840,1081.0,Lee,Alabama,US,32.601549,-85.351322,...,9715,9779,9800,9856,9928,10047,10159,10320,10450,10508


In [40]:
date_columns = jh_hist_uscounties_cases.iloc[:,11:].columns

In [41]:
jh_hist_usstates_cases = pd.pivot_table(jh_hist_uscounties_cases, index=['Province_State'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_usstates_cases = jh_hist_usstates_cases.transpose()
jh_hist_usstates_cases.index = pd.to_datetime(jh_hist_usstates_cases.index)
jh_hist_usstates_cases = jh_hist_usstates_cases.sort_index()

In [42]:
jh_hist_usstates_cases.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-12-31,361226,46986,0,520207,225138,2291007,334097,185708,57456,49,...,586802,1766307,276612,7412,2031,349584,246752,85334,520438,44409
2021-01-01,365747,46986,0,530267,229442,2336201,337161,185708,58064,49,...,586802,1779595,276612,7412,2036,354766,246752,87820,522523,44409
2021-01-02,369458,47801,0,539150,231442,2388557,339172,190120,58873,49,...,604132,1793453,281654,7689,2041,358755,246752,89327,523652,44573


##### 1B-8: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [43]:
jh_hist_uscounties_cases = pd.pivot_table(jh_hist_uscounties_cases, index=['Province_State', 'Admin2'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_uscounties_cases = jh_hist_uscounties_cases.transpose()
jh_hist_uscounties_cases.index = pd.to_datetime(jh_hist_uscounties_cases.index)
jh_hist_uscounties_cases = jh_hist_uscounties_cases.sort_index()

In [44]:
jh_hist_uscounties_cases.tail(3)

Province_State Alabama                                                      \
Admin2         Autauga Baldwin Barbour  Bibb Blount Bullock Butler Calhoun   
2020-12-31        4190   13601    1514  1834   4641     859   1508    9494   
2021-01-01        4239   13823    1517  1854   4693     888   1522    9584   
2021-01-02        4268   13955    1528  1863   4729     892   1530    9692   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2020-12-31         2341     1414  ...    2021    440     2543      597   
2021-01-01         2366     1429  ...    2021    440     2543      597   
2021-01-02         2386     1435  ...    2035    441     2555      598   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2020-12-31           2966  2138  1558          0      781    476  
2021-01-01           2966  2138  1558          0      781    476  
2021-01-02           2992  2145  1569          0      782    477  

[3 rows x 3334 columns]

##### 1B-9: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **US State**

In [45]:
jh_hist_usstates_deaths = pd.pivot_table(jh_hist_uscounties_deaths, index=['Province_State'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_usstates_deaths = jh_hist_usstates_deaths.transpose()
jh_hist_usstates_deaths.index = pd.to_datetime(jh_hist_usstates_deaths.index)
jh_hist_usstates_deaths = jh_hist_usstates_deaths.sort_index()

In [46]:
jh_hist_usstates_deaths.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-12-31,4827,206,0,8864,3676,25897,4814,5995,926,0,...,6907,27944,1269,136,23,5032,3461,1338,5242,438
2021-01-01,4872,206,0,9015,3711,26286,4873,5995,930,0,...,6907,28253,1269,136,23,5081,3461,1361,5254,438
2021-01-02,4872,215,0,9061,3729,26542,4912,6099,930,0,...,6970,28338,1294,139,23,5117,3461,1373,5256,438


##### 1B-10: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **US County**

In [47]:
jh_hist_uscounties_deaths = pd.pivot_table(jh_hist_uscounties_deaths, index=['Province_State', 'Admin2'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_uscounties_deaths = jh_hist_uscounties_deaths.transpose()
jh_hist_uscounties_deaths.index = pd.to_datetime(jh_hist_uscounties_deaths.index)
jh_hist_uscounties_deaths = jh_hist_uscounties_deaths.sort_index()

In [48]:
jh_hist_uscounties_deaths.tail(3)

Province_State Alabama                                                     \
Admin2         Autauga Baldwin Barbour Bibb Blount Bullock Butler Calhoun   
2020-12-31          48     161      32   46     63      22     45     156   
2021-01-01          50     169      33   46     63      22     45     157   
2021-01-02          50     169      33   46     63      22     45     157   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2020-12-31           63       22  ...      10      8       17        6   
2021-01-01           63       22  ...      10      8       17        6   
2021-01-02           63       22  ...      10      8       17        6   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2020-12-31             16     4     7          0       19      2  
2021-01-01             16     4     7          0       19      2  
2021-01-02             16     4     7          0       19      2  

[3 rows x 3334 columns]

#### 1C: Export the data

##### 1C-1: Export the JH live global data, grouped / subtotalled by **Country**

In [49]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_countries.csv')
jh_live_countries.to_csv(filename, index=True)

##### 1C-2: Export the JH live global data, grouped / subtotalled by **US State**

In [50]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_usstates.csv')
jh_live_usstates.to_csv(filename, index=True)

##### 1C-3: Export the JH live global data, grouped / subtotalled by **US County**

In [51]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_uscounties.csv')
jh_live_uscounties.to_csv(filename, index=True)

##### 1C-4: Export the JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [52]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases.csv')
jh_hist_countries_cases.to_csv(filename, index=True)

##### 1C-5: Export the JH historic **deaths**, grouped / subtotalled by **Country**

In [53]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths.csv')
jh_hist_countries_deaths.to_csv(filename, index=True)

##### 1C-6: Export the JH historic **recoveries**, grouped / subtotalled by **Country**

In [54]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered.csv')
jh_hist_countries_recovered.to_csv(filename, index=True)

##### 1C-7: Export the JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [55]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases.csv')
jh_hist_usstates_cases.to_csv(filename, index=True)

##### 1C-8: Export the JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [56]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases.csv')
jh_hist_uscounties_cases.to_csv(filename, index=True)

##### 1C-9: Export the JH historic **deaths**, grouped / subtotalled by **US State**

In [57]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths.csv')
jh_hist_usstates_deaths.to_csv(filename, index=True)

##### 1C-10: Export the JH historic **deaths**, grouped / subtotalled by **US County**

In [58]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths.csv')
jh_hist_uscounties_deaths.to_csv(filename, index=True)

#### 1D: Calculate daily % change for each historic dataset (match numbering sequence, export as well)

##### 1D-4: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [59]:
jh_hist_countries_cases_pct = jh_hist_countries_cases.pct_change()

In [60]:
jh_hist_countries_cases_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2020-12-31,0.002354,0.010203,0.003011,0.008268,0.006883,0.006329,0.007179,0.003342,0.000704,0.008139,...,0.022961,0.034578,0.000974,0.0,0.003863,0.006181,0.009273,0.000954,0.012853,0.017761
2021-01-01,0.000000,0.000000,0.002881,0.008448,0.000855,0.000000,0.002510,0.002064,0.001231,0.005809,...,0.021415,0.033161,0.001583,0.0,0.002871,0.006143,0.008833,0.000953,0.013124,0.015649
2021-01-02,0.000000,0.011575,0.002623,0.006037,0.002277,0.000000,0.003216,0.000376,0.000843,0.003833,...,0.022690,0.026426,0.000000,0.0,0.001747,0.005427,0.007642,0.000000,0.011097,0.028898


In [61]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases_pct.csv')
jh_hist_countries_cases_pct.to_csv(filename, index=True)

##### 1D-5: Daily % change JH historic **deaths**, grouped / subtotalled by **Country**

In [62]:
jh_hist_countries_deaths_pct = jh_hist_countries_deaths.pct_change()

In [63]:
jh_hist_countries_deaths_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2020-12-31,0.004585,0.005963,0.001818,0.0,0.000000,0.0,0.001900,0.005700,0.0,0.011872,...,0.013282,0.040230,0.0,NaN,0.002927,0.0,0.021152,0.0,0.005181,0.008333
2021-01-01,0.000000,0.000000,0.002177,0.0,0.000000,0.0,0.001711,0.001771,0.0,0.006268,...,0.008353,0.066298,0.0,NaN,0.001946,0.0,0.012857,0.0,0.005155,0.016529
2021-01-02,0.000000,0.007621,0.002534,0.0,0.004938,0.0,0.001293,0.002829,0.0,0.002236,...,0.005994,0.041451,0.0,NaN,0.001942,0.0,0.019746,0.0,0.005128,0.021680


In [64]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths_pct.csv')
jh_hist_countries_deaths_pct.to_csv(filename, index=True)

##### 1D-6: Daily % change Export the JH historic **recoveries**, grouped / subtotalled by **Country**

In [65]:
jh_hist_countries_recovered_pct = jh_hist_countries_recovered.pct_change()

In [66]:
jh_hist_countries_recovered_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2020-12-31,0.002764,0.013530,0.004069,0.006501,0.017037,0.0,0.00000,0.006747,0.000039,0.005928,...,0.012006,0.036319,0.002059,0.0,0.003545,0.001512,0.017859,0.001437,0.007016,0.008607
2021-01-01,0.000000,0.000000,0.003992,0.004171,0.009236,0.0,0.00000,0.003880,0.000311,0.005854,...,0.036503,0.038759,0.004016,0.0,0.002956,0.000000,0.014874,0.001435,0.006056,0.008622
2021-01-02,0.000000,0.021377,0.003205,0.000000,0.003858,0.0,0.01431,0.001988,0.000310,0.006035,...,0.011446,0.043031,0.000000,0.0,0.001715,0.009057,0.012419,0.000000,0.009002,0.001410


In [67]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered_pct.csv')
jh_hist_countries_recovered_pct.to_csv(filename, index=True)

##### 1D-7: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [68]:
jh_hist_usstates_cases_pct = jh_hist_usstates_cases.pct_change()

In [69]:
jh_hist_usstates_cases_pct.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-12-31,0.012348,0.010821,NaN,0.015060,0.012175,0.013256,0.009787,0.011135,0.015195,0.0,...,0.010318,0.009174,0.017180,0.018692,0.010448,0.015214,0.018248,0.013167,0.008159,0.006254
2021-01-01,0.012516,0.000000,NaN,0.019338,0.019117,0.019727,0.009171,0.000000,0.010582,0.0,...,0.000000,0.007523,0.000000,0.000000,0.002462,0.014823,0.000000,0.029133,0.004006,0.000000
2021-01-02,0.010146,0.017346,NaN,0.016752,0.008717,0.022411,0.005965,0.023758,0.013933,0.0,...,0.029533,0.007787,0.018228,0.037372,0.002456,0.011244,0.000000,0.017160,0.002161,0.003693


In [70]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases_pct.csv')
jh_hist_usstates_cases_pct.to_csv(filename, index=True)

##### 1D-8: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [71]:
jh_hist_uscounties_cases_pct = jh_hist_uscounties_cases.pct_change()

In [72]:
jh_hist_uscounties_cases_pct.tail(3)

Province_State   Alabama                                                    \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   Bullock   
2020-12-31      0.006244  0.015606  0.014745  0.009356  0.012435  0.015366   
2021-01-01      0.011695  0.016322  0.001982  0.010905  0.011204  0.033760   
2021-01-02      0.006841  0.009549  0.007251  0.004854  0.007671  0.004505   

Province_State                                          ...   Wyoming  \
Admin2            Butler   Calhoun  Chambers  Cherokee  ...      Park   
2020-12-31      0.010047  0.007000  0.013420  0.007840  ...  0.018136   
2021-01-01      0.009284  0.009480  0.010679  0.010608  ...  0.000000   
2021-01-02      0.005256  0.011269  0.008453  0.004199  ...  0.006927   

Province_State                                                               \
Admin2            Platte  Sheridan  Sublette Sweetwater     Teton     Uinta   
2020-12-31      0.013825  0.008727  0.008446   0.007815  0.008015  0.011688   
2021-01-01      0.000000  0.000000  0.000000   0.000000  0.000000  0.000000   
2021-01-02      0.002273  0.004719  0.001675   0.008766  0.003274  0.007060   

Province_State                                 
Admin2         Unassigned  Washakie    Weston  
2020-12-31            NaN  0.016927  0.002105  
2021-01-01            NaN  0.000000  0.000000  
2021-01-02            NaN  0.001280  0.002101  

[3 rows x 3334 columns]

In [73]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases_pct.csv')
jh_hist_uscounties_cases_pct.to_csv(filename, index=True)

##### 1D-9: Daily % change JH historic **deaths**, grouped / subtotalled by **US State**

In [74]:
jh_hist_usstates_deaths_pct = jh_hist_usstates_deaths.pct_change()

In [75]:
jh_hist_usstates_deaths_pct.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-12-31,0.011102,0.014778,NaN,0.016747,0.010723,0.022223,0.013474,0.005198,0.005429,NaN,...,0.014244,0.012464,0.010350,0.014925,0.0,0.009631,0.011988,0.015175,0.009630,0.081481
2021-01-01,0.009323,0.000000,NaN,0.017035,0.009521,0.015021,0.012256,0.000000,0.004320,NaN,...,0.000000,0.011058,0.000000,0.000000,0.0,0.009738,0.000000,0.017190,0.002289,0.000000
2021-01-02,0.000000,0.043689,NaN,0.005103,0.004850,0.009739,0.008003,0.017348,0.000000,NaN,...,0.009121,0.003009,0.019701,0.022059,0.0,0.007085,0.000000,0.008817,0.000381,0.000000


In [76]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths_pct.csv')
jh_hist_usstates_deaths_pct.to_csv(filename, index=True)

##### 1D-10: Daily % change JH historic **deaths**, grouped / subtotalled by **US County**

In [77]:
jh_hist_uscounties_deaths_pct = jh_hist_uscounties_deaths.pct_change()

In [78]:
jh_hist_uscounties_deaths_pct.tail(3)

Province_State   Alabama                                                \
Admin2           Autauga   Baldwin  Barbour Bibb Blount Bullock Butler   
2020-12-31      0.000000  0.006250  0.00000  0.0    0.0     0.1    0.0   
2021-01-01      0.041667  0.049689  0.03125  0.0    0.0     0.0    0.0   
2021-01-02      0.000000  0.000000  0.00000  0.0    0.0     0.0    0.0   

Province_State                               ...   Wyoming                  \
Admin2           Calhoun  Chambers Cherokee  ...      Park Platte Sheridan   
2020-12-31      0.006452  0.016129      0.0  ...  0.428571    0.0   0.0625   
2021-01-01      0.006410  0.000000      0.0  ...  0.000000    0.0   0.0000   
2021-01-02      0.000000  0.000000      0.0  ...  0.000000    0.0   0.0000   

Province_State                                                                 
Admin2         Sublette Sweetwater     Teton Uinta Unassigned Washakie Weston  
2020-12-31          0.2   0.066667  0.333333   0.0        NaN   0.1875    0.0  
2021-01-01          0.0   0.000000  0.000000   0.0        NaN   0.0000    0.0  
2021-01-02          0.0   0.000000  0.000000   0.0        NaN   0.0000    0.0  

[3 rows x 3334 columns]

In [79]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths_pct.csv')
jh_hist_uscounties_deaths_pct.to_csv(filename, index=True)

#### 1E: Calculate exponential weighted moving average (3 day) for each historic dataset (match numbering sequence, export as well)

This is useful to see which countries / states / counties are becoming hot spots or which are cooling down.  
Exponential weighted moving averages (EWMA) blend the most recent x days, using a heavier weight for more recent data.  
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.ewm.html  
Using the EWMA gives the best balance of minimizing erratic daily changes by using a moving avg, while giving more credence to recent data vs a simple moving avg.  

##### 1E-4: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [80]:
jh_hist_countries_cases_pct_ewma = jh_hist_countries_cases_pct.ewm(span=3).mean()

In [81]:
jh_hist_countries_cases_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2020-12-31,0.002085,0.009800,0.003280,0.007535,0.005370,0.00444,0.006835,0.003132,0.000780,0.007093,...,0.021664,0.033314,0.001009,0.0,0.003128,0.004234,0.009270,0.000695,0.012301,0.017042
2021-01-01,0.001043,0.004900,0.003081,0.007992,0.003112,0.00222,0.004672,0.002598,0.001006,0.006451,...,0.021540,0.033237,0.001296,0.0,0.003000,0.005189,0.009051,0.000824,0.012712,0.016345
2021-01-02,0.000521,0.008237,0.002852,0.007014,0.002694,0.00111,0.003944,0.001487,0.000925,0.005142,...,0.022115,0.029832,0.000648,0.0,0.002373,0.005308,0.008347,0.000412,0.011905,0.022622


In [82]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases_pct_ewma.csv')
jh_hist_countries_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-5: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **Country**

In [83]:
jh_hist_countries_deaths_pct_ewma = jh_hist_countries_deaths_pct.ewm(span=3).mean()

In [84]:
jh_hist_countries_deaths_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2020-12-31,0.003546,0.005597,0.002189,0.001570,0.002059,4.768372e-07,0.002720,0.005299,0.000034,0.013438,...,0.011518,0.039696,0.000459,NaN,0.003200,2.991983e-38,0.018355,0.000669,0.003881,0.008452
2021-01-01,0.001773,0.002798,0.002183,0.000785,0.001029,2.384186e-07,0.002216,0.003535,0.000017,0.009853,...,0.009936,0.052997,0.000229,NaN,0.002573,1.495992e-38,0.015606,0.000335,0.004518,0.012490
2021-01-02,0.000886,0.005209,0.002359,0.000392,0.002984,1.192093e-07,0.001754,0.003182,0.000009,0.006044,...,0.007965,0.047224,0.000115,NaN,0.002257,7.479959e-39,0.017676,0.000167,0.004823,0.017085


In [85]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths_pct_ewma.csv')
jh_hist_countries_deaths_pct_ewma.to_csv(filename, index=True)

##### 1E-6: EWMA of Daily % change Export the JH historic **recovered**, grouped / subtotalled by **Country**

In [86]:
jh_hist_countries_recovered_pct_ewma = jh_hist_countries_recovered_pct.ewm(span=3).mean()

In [87]:
jh_hist_countries_recovered_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2020-12-31,0.003098,0.014675,0.004496,0.005393,0.018711,0.002167,0.002211,0.006680,0.000160,0.006550,...,0.018168,0.039656,0.002127,0.0,0.002961,0.003193,0.017595,0.001891,0.006566,0.011621
2021-01-01,0.001549,0.007337,0.004244,0.004782,0.013973,0.001083,0.001105,0.005280,0.000235,0.006202,...,0.027336,0.039207,0.003072,0.0,0.002959,0.001597,0.016235,0.001663,0.006311,0.010122
2021-01-02,0.000775,0.014357,0.003725,0.002391,0.008916,0.000542,0.007708,0.003634,0.000273,0.006118,...,0.019391,0.041119,0.001536,0.0,0.002337,0.005327,0.014327,0.000831,0.007657,0.005766


In [88]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered_pct_ewma.csv')
jh_hist_countries_recovered_pct_ewma.to_csv(filename, index=True)

##### 1E-7: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [89]:
jh_hist_usstates_cases_pct_ewma = jh_hist_usstates_cases_pct.ewm(span=3).mean()

In [90]:
jh_hist_usstates_cases_pct_ewma.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-12-31,0.011982,0.008597,NaN,0.012591,0.012136,0.014598,0.008083,0.011578,0.012248,1.711289e-88,...,0.010897,0.010593,0.013089,0.014978,0.009248,0.013265,0.012647,0.014351,0.006783,0.005921
2021-01-01,0.012249,0.004298,NaN,0.015965,0.015626,0.017162,0.008627,0.005789,0.011415,8.556445e-89,...,0.005448,0.009058,0.006544,0.007489,0.005855,0.014044,0.006324,0.021742,0.005395,0.002960
2021-01-02,0.011198,0.010822,NaN,0.016358,0.012172,0.019786,0.007296,0.014773,0.012674,4.278223e-89,...,0.017491,0.008423,0.012386,0.022431,0.004155,0.012644,0.003162,0.019451,0.003778,0.003327


In [91]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases_pct_ewma.csv')
jh_hist_usstates_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-8: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [92]:
jh_hist_uscounties_cases_pct_ewma = jh_hist_uscounties_cases_pct.ewm(span=3).mean()

In [93]:
jh_hist_uscounties_cases_pct_ewma.tail(3)

Province_State   Alabama                                                    \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   Bullock   
2020-12-31      0.008974  0.015219  0.017267  0.011305  0.010920  0.013270   
2021-01-01      0.010334  0.015771  0.009624  0.011105  0.011062  0.023515   
2021-01-02      0.008588  0.012660  0.008438  0.007980  0.009367  0.014010   

Province_State                                          ...   Wyoming  \
Admin2            Butler   Calhoun  Chambers  Cherokee  ...      Park   
2020-12-31      0.009479  0.007228  0.013827  0.008019  ...  0.014606   
2021-01-01      0.009381  0.008354  0.012253  0.009314  ...  0.007303   
2021-01-02      0.007319  0.009811  0.010353  0.006756  ...  0.007115   

Province_State                                                               \
Admin2            Platte  Sheridan  Sublette Sweetwater     Teton     Uinta   
2020-12-31      0.013645  0.007516  0.006466   0.007196  0.006485  0.010844   
2021-01-01      0.006823  0.003758  0.003233   0.003598  0.003242  0.005422   
2021-01-02      0.004548  0.004238  0.002454   0.006182  0.003258  0.006241   

Province_State                                 
Admin2         Unassigned  Washakie    Weston  
2020-12-31           -1.0  0.011687  0.004796  
2021-01-01           -1.0  0.005844  0.002398  
2021-01-02           -1.0  0.003562  0.002249  

[3 rows x 3334 columns]

In [94]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases_pct_ewma.csv')
jh_hist_uscounties_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-9: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **US State**

In [95]:
jh_hist_usstates_deaths_pct_ewma = jh_hist_usstates_deaths_pct.ewm(span=3).mean()

In [96]:
jh_hist_usstates_deaths_pct_ewma.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-12-31,0.008663,0.010705,NaN,0.013705,0.011585,0.018579,0.012221,0.005992,0.009435,NaN,...,0.014355,0.010644,0.011561,0.018994,2.403082e-19,0.009939,0.016858,0.017149,0.009487,0.046166
2021-01-01,0.008993,0.005352,NaN,0.015370,0.010553,0.016800,0.012239,0.002996,0.006877,NaN,...,0.007177,0.010851,0.005781,0.009497,1.201541e-19,0.009839,0.008429,0.017169,0.005888,0.023083
2021-01-02,0.004496,0.024521,NaN,0.010236,0.007702,0.013270,0.010121,0.010172,0.003439,NaN,...,0.008149,0.006930,0.012741,0.015778,6.007704e-20,0.008462,0.004214,0.012993,0.003134,0.011542


In [97]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths_pct_ewma.csv')
jh_hist_usstates_deaths_pct_ewma.to_csv(filename, index=True)

##### 1E-10: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **US County**

In [98]:
jh_hist_uscounties_deaths_pct_ewma = jh_hist_uscounties_deaths_pct.ewm(span=3).mean()

In [99]:
jh_hist_uscounties_deaths_pct_ewma.tail(3)

Province_State   Alabama                                                  \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount Bullock   
2020-12-31      0.006089  0.013073  0.000008  0.023811  0.021735  0.0500   
2021-01-01      0.023878  0.031381  0.015629  0.011905  0.010868  0.0250   
2021-01-02      0.011939  0.015691  0.007815  0.005953  0.005434  0.0125   

Province_State                                              ...   Wyoming  \
Admin2                Butler   Calhoun  Chambers  Cherokee  ...      Park   
2020-12-31      1.409872e-06  0.013493  0.012184 -0.010872  ...  0.214286   
2021-01-01      7.049360e-07  0.009952  0.006092 -0.005436  ...  0.107143   
2021-01-02      3.524680e-07  0.004976  0.003046 -0.002718  ...  0.053571   

Province_State                                                        \
Admin2                Platte  Sheridan Sublette Sweetwater     Teton   
2020-12-31      6.810039e-11  0.035421    0.100   0.033334  0.197917   
2021-01-01      3.405019e-11  0.017711    0.050   0.016667  0.098958   
2021-01-02      1.702510e-11  0.008855    0.025   0.008333  0.049479   

Province_State                                                   
Admin2                 Uinta Unassigned  Washakie        Weston  
2020-12-31      2.629434e-08  -0.501961  0.114672  4.656612e-10  
2021-01-01      1.314717e-08  -0.501961  0.057336  2.328306e-10  
2021-01-02      6.573585e-09  -0.501961  0.028668  1.164153e-10  

[3 rows x 3334 columns]

In [100]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths_pct_ewma.csv')
jh_hist_uscounties_deaths_pct_ewma.to_csv(filename, index=True)

### Dataset 2: US States - Covid Tracking Project

The Covid Tracking Project stores data in both API (JSON) and CSV formats.  We'll be loading the CSV files into Pandas here.  
https://covidtracking.com/api/  

States current - /api/v1/states/current.csv | [CSV](https://covidtracking.com/api/v1/states/current.csv)  
States historic - /api/v1/states/daily.csv | [CSV](https://covidtracking.com/api/v1/states/daily.csv)  
States info - /api/v1/states/info.csv | [CSV](https://covidtracking.com/api/v1/states/info.csv)   

#### 2A: Pull the Data from CT's API, immediately export as 'orig'

In [101]:
ct_live_usstates = pd.read_csv("https://covidtracking.com/api/v1/states/current.csv")
ct_live_usstates.head()

,date,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
0,20210102,AK,46262,NaN,1244087,NaN,totalTestsViral,1290349,77.0,1030.0,...,1290349,9,7,8a884e8ed2e7092f03851a18d2fb2bdc26a80496,0,0,0,0,0,NaN
1,20210102,AL,369458,71705.0,1593715,NaN,totalTestsPeopleViral,1891468,2815.0,34373.0,...,1963173,0,0,3b521f3cb9b6ad6792a54211f5741a54186e1ffe,0,0,0,0,0,NaN
2,20210102,AR,231442,40276.0,1888622,NaN,totalTestsViral,2079788,1216.0,11487.0,...,2120064,18,37,a2a19dc4deb4dff755942d83f312742303a02c9f,0,0,0,0,0,NaN
3,20210102,AS,0,NaN,2140,NaN,totalTestsViral,2140,NaN,NaN,...,2140,0,0,2c1f80bcf1772c718587b84a2cfcd15c9bc94c7f,0,0,0,0,0,NaN
4,20210102,AZ,539150,27103.0,2393336,NaN,totalTestsViral,5218721,4484.0,39439.0,...,2932486,46,593,2f8dff6a903083bdeae74643f35e28c842438e23,0,0,0,0,0,NaN


In [102]:
filename = os.path.join(os.getcwd(), 'Datasets/CT/orig/ct_live_usstates_orig.csv')
ct_live_usstates.to_csv(filename, index=False)

In [103]:
ct_hist_usstates = pd.read_csv("https://covidtracking.com/api/v1/states/daily.csv")
ct_hist_usstates.head()

/Users/Brian/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3147: DtypeWarning: Columns (15) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,date,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
0,20210102,AK,46262.0,NaN,1244087.0,NaN,totalTestsViral,1290349.0,77.0,1030.0,...,1290349,9,7,8a884e8ed2e7092f03851a18d2fb2bdc26a80496,0,0,0,0,0,NaN
1,20210102,AL,369458.0,71705.0,1593715.0,NaN,totalTestsPeopleViral,1891468.0,2815.0,34373.0,...,1963173,0,0,3b521f3cb9b6ad6792a54211f5741a54186e1ffe,0,0,0,0,0,NaN
2,20210102,AR,231442.0,40276.0,1888622.0,NaN,totalTestsViral,2079788.0,1216.0,11487.0,...,2120064,18,37,a2a19dc4deb4dff755942d83f312742303a02c9f,0,0,0,0,0,NaN
3,20210102,AS,0.0,NaN,2140.0,NaN,totalTestsViral,2140.0,NaN,NaN,...,2140,0,0,2c1f80bcf1772c718587b84a2cfcd15c9bc94c7f,0,0,0,0,0,NaN
4,20210102,AZ,539150.0,27103.0,2393336.0,NaN,totalTestsViral,5218721.0,4484.0,39439.0,...,2932486,46,593,2f8dff6a903083bdeae74643f35e28c842438e23,0,0,0,0,0,NaN


In [104]:
filename = os.path.join(os.getcwd(), 'Datasets/CT/orig/ct_hist_usstates_orig.csv')
ct_hist_usstates.to_csv(filename, index=False)

#### 2B: Change the dates to datetime format, make them the index for both datasets

In [105]:
ct_live_usstates['date'] = ct_live_usstates['date'].map(lambda d : str(d)[:4] + '-' + str(d)[4:6] + '-' + str(d)[6:])
ct_live_usstates['date'] = pd.to_datetime(ct_live_usstates['date'])
ct_live_usstates.set_index('date', inplace=True)
ct_live_usstates.head()

,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
date,,,,,,,,,,,,,,,,,,,,,
2021-01-02,AK,46262,NaN,1244087,NaN,totalTestsViral,1290349,77.0,1030.0,NaN,...,1290349,9,7,8a884e8ed2e7092f03851a18d2fb2bdc26a80496,0,0,0,0,0,NaN
2021-01-02,AL,369458,71705.0,1593715,NaN,totalTestsPeopleViral,1891468,2815.0,34373.0,NaN,...,1963173,0,0,3b521f3cb9b6ad6792a54211f5741a54186e1ffe,0,0,0,0,0,NaN
2021-01-02,AR,231442,40276.0,1888622,NaN,totalTestsViral,2079788,1216.0,11487.0,393.0,...,2120064,18,37,a2a19dc4deb4dff755942d83f312742303a02c9f,0,0,0,0,0,NaN
2021-01-02,AS,0,NaN,2140,NaN,totalTestsViral,2140,NaN,NaN,NaN,...,2140,0,0,2c1f80bcf1772c718587b84a2cfcd15c9bc94c7f,0,0,0,0,0,NaN
2021-01-02,AZ,539150,27103.0,2393336,NaN,totalTestsViral,5218721,4484.0,39439.0,1074.0,...,2932486,46,593,2f8dff6a903083bdeae74643f35e28c842438e23,0,0,0,0,0,NaN


In [106]:
ct_hist_usstates['date'] = ct_hist_usstates['date'].map(lambda d : str(d)[:4] + '-' + str(d)[4:6] + '-' + str(d)[6:])
ct_hist_usstates['date'] = pd.to_datetime(ct_hist_usstates['date'])
ct_hist_usstates.set_index('date', inplace=True)
ct_hist_usstates.head()

,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
date,,,,,,,,,,,,,,,,,,,,,
2021-01-02,AK,46262.0,NaN,1244087.0,NaN,totalTestsViral,1290349.0,77.0,1030.0,NaN,...,1290349,9,7,8a884e8ed2e7092f03851a18d2fb2bdc26a80496,0,0,0,0,0,NaN
2021-01-02,AL,369458.0,71705.0,1593715.0,NaN,totalTestsPeopleViral,1891468.0,2815.0,34373.0,NaN,...,1963173,0,0,3b521f3cb9b6ad6792a54211f5741a54186e1ffe,0,0,0,0,0,NaN
2021-01-02,AR,231442.0,40276.0,1888622.0,NaN,totalTestsViral,2079788.0,1216.0,11487.0,393.0,...,2120064,18,37,a2a19dc4deb4dff755942d83f312742303a02c9f,0,0,0,0,0,NaN
2021-01-02,AS,0.0,NaN,2140.0,NaN,totalTestsViral,2140.0,NaN,NaN,NaN,...,2140,0,0,2c1f80bcf1772c718587b84a2cfcd15c9bc94c7f,0,0,0,0,0,NaN
2021-01-02,AZ,539150.0,27103.0,2393336.0,NaN,totalTestsViral,5218721.0,4484.0,39439.0,1074.0,...,2932486,46,593,2f8dff6a903083bdeae74643f35e28c842438e23,0,0,0,0,0,NaN
